## Вопросно-ответная система

### Сбор данных

Для выполнения данного проекта были собраны статьи из энциклопедии http://www.krugosvet.ru/, посвященные странам мира. Всего было собрано 15 статей по 15 странам. Данные статьи отличаются большим размером (до 40 страниц) и охватывают все сферы от географии до экономики и политики.

In [1]:
import requests
from bs4 import BeautifulSoup
import re

Написан веб-краулер, который автоматически собирает абзацы статей по страницам сайта. Реализован краулер с помощью библиотеки request, парсинг страниц осуществлен с помощью BeautifulSoup. Каждый обзац обрабатывается: убираются теги и спецсимволы.

In [2]:
def clean(string):
    string = re.sub('<[^>]*>', '', string)
    string = re.sub('&(.*);', '', string)
    return string
    
def get_county_info(link):
    data = []
    for i in range(50):
        text = requests.get(link + '.html?page=0,' + str(i)).text
        paragraphs = BeautifulSoup(text, 'lxml').find_all('p')[1:]
        if len(paragraphs) > 0:
            paragraphs = [clean(p.text) for p in paragraphs]
            paragraphs = [p for p in paragraphs if len(p)]
            data += paragraphs
        else:
            return data
    return data

Собираем данные по странам:

In [3]:
links = ['AVSTRALIYA', 'AVSTRIYA', 'BELGIYA', 'BRAZILIYA', 'SOEDINENNIE_SHTATI_AMERIKI',
         'GERMANIYA', 'KITA', 'UKRAINA', 'FRANTSIYA', 'INDIYA', 'ITALIYA', 'ISPANIYA', 'KANADA']
links = ['http://www.krugosvet.ru/enc/strany_mira/'+ l for l in links]
links += ["http://www.krugosvet.ru/enc/Earth_sciences/geografiya/ROSSIYA_ROSSISKAYA_FEDERATSIYA_RF"]
links += ["http://www.krugosvet.ru/enc/Earth_sciences/geografiya/SOEDINENNOE_KOROLEVSTVO_VELIKOBRITANII_I_SEVERNO_IRLANDII"]
ru_countries = ['Австралия', 'Австрия', 'Бельгия', 'Бразилия', 'США', 'Германия', 'Китай', 'Украина', 
                'Франция', 'Индия', 'Италия', 'Испания', 'Канада', 'Россия', 'Великобритания']

all_data = []
for i in range(15):
    newdata = get_county_info(links[i])
    all_data += newdata
    print(ru_countries[i], 'данные собраны. Количество абзацев', len(newdata))

Австралия данные собраны. Количество абзацев 442
Австрия данные собраны. Количество абзацев 293
Бельгия данные собраны. Количество абзацев 196
Бразилия данные собраны. Количество абзацев 318
США данные собраны. Количество абзацев 653
Германия данные собраны. Количество абзацев 452
Китай данные собраны. Количество абзацев 453
Украина данные собраны. Количество абзацев 173
Франция данные собраны. Количество абзацев 407
Индия данные собраны. Количество абзацев 366
Италия данные собраны. Количество абзацев 447
Испания данные собраны. Количество абзацев 229
Канада данные собраны. Количество абзацев 214
Россия данные собраны. Количество абзацев 693
Великобритания данные собраны. Количество абзацев 374


Получаем итоговый текст, соединяя абзацы в один.

In [4]:
final_text = ' '.join(all_data)

Данные можно взять в приложенном файле

In [ ]:
final_text = open('data.txt').read()

### Поиск предложений с помощью модели word2vec

Прежде чем выделять ответ на вопрос, необходимо определить множество релевантных фрагментов. Мы хотим находить самое похожее на вопрос предложение. Можно было бы просто найти предложение с наибольшим пересечением по словам с вопросом. Но так как вопрос может быть перефразирован, воспользуемся синонимами из предобученной модели word2vec (НКРЯ и Википедия за ноябрь 2016).

Создан класс QA_word2vec, в котором реализованы методы для предобработки текста и последующего поиска предложений. Модель поиска состоит из следующих этапов:

- Для вопроса производится стемминг с помощью библиотеки pymystem3. Каждое слово приводится к начальной форме и добавляется постфикс части речи.
- Для каждого слова из вопроса, которое не является стоп-словом, ищется топ-10 синонимов из словаря с помощью функции  wv.most_similar(). Она также возвращает число, обозначающее похожесть двух данных слов в модели word2vec.
- Для каждого предложения из базы данных вычисляется его score: для каждого слова из вопроса ищем в предложении либо само слово, либо наиболее похожее на него из ранее полученного топ-10, мера схожести найденного слова (1, если найдено само слово) добавляется к score предложения. Итоговая сумма делится на число слов в вопросе, получаем рейтинг предложения в процентах.


In [5]:
from nltk.tokenize import sent_tokenize
from pymystem3 import Mystem
from gensim.models.keyedvectors import KeyedVectors
from stop_words import get_stop_words
import numpy as np

Класс с необходимыми методами

In [6]:
class QA_word2vec:
    
    def __init__(self, text):
        self.text = text
        self.w2v_model = KeyedVectors.load_word2vec_format('ruwikiruscorpora_0_300_20.bin.gz', binary=True,
                                                           limit=500000)
        self.mystem2uni = {'A': 'ADJ', 'ADV': 'ADV', 'ADVPRO': 'ADV', 'ANUM': 'ADJ', 'APRO': 'DET', 'COM': 'ADJ',
                           'CONJ': 'SCONJ', 'INTJ': 'INTJ', 'NONLEX': 'X', 'NUM': 'NUM', 'PART': 'PART',
                           'PR': 'ADP', 'S': 'NOUN', 'SPRO': 'PRON', 'UNKN': 'X', 'V': 'VERB'}
        self.ms = Mystem()
        # Text preprocessing: sentence tokenization and normalization
        self.sents = np.array(sent_tokenize(self.text))
        self.norm_sents = []
        for sent in self.sents:
            self.norm_sents.append(self.normalize_sent(sent))
        self.norm_sents = np.array(self.norm_sents)
    
    def normalize_sent(self, sent):
        # Sentence normalization
        analysis = self.ms.analyze(sent)
        norm_sent = []
        for word in analysis:
            if 'analysis' in word and len(word['analysis']):
                word_norm = word['analysis'][0]['lex']
                form = self.mystem2uni[word['analysis'][0]['gr'].split('=')[0].split(',')[0]]
                norm_sent.append(word_norm + '_' + form)
        return norm_sent
    
    def score_sentence(self, sent, sim_words):
        # for each word in the question we find the most similar word 
        # in a sentence and add its score to the total sentence score
        sent_score = 0
        for words in sim_words:
            best_score = 0
            for word in words:
                if word[0] in sent:
                    best_score = max(best_score, word[1])
            sent_score += best_score
        return sent_score
                    
    def get_relevant_sentences(self, question, n=10):
        # Question normalization and stop-words filtering
        norm_question = [word for word in self.normalize_sent(question) \
                         if word.split('_')[0] not in get_stop_words('russian')]
        # For each word find 10 similar words
        sim_words = []
        for word in norm_question:
            try:
                # each element of sim_words is a list of tupples (word, score)
                sim_words.append([(word, 1)] + self.w2v_model.wv.most_similar(word))
            except:
                pass
        # Calculate scores for each word in every sentence
        sent_scores = np.zeros(self.sents.shape[0])
        for i, sent in enumerate(self.norm_sents):
            sent_scores[i] = self.score_sentence(sent, sim_words)
        # Return n best sentences and their scores
        best_sort = np.argsort(-sent_scores)[:n]
        return self.sents[best_sort], sent_scores[best_sort] / len(norm_question)

Инициализируем объект класса, сразу происходит нормализация всех предложений в базе. Это занимает около минуты.

In [7]:
searcher = QA_word2vec(final_text)

Итоговое число предложений в базе:

In [8]:
len(searcher.sents)

28019

Протестируем наш поиск

In [9]:
def print_search(question, n):
    sents, scores = searcher.get_relevant_sentences(question, n=n)
    print('\033[1mQuestion\033[0m', question)
    for i in range(n):
        print('\033[1mTop' + str(i+1) + '\033[0m', sents[i], scores[i])

Для вопросов, части которых в прямом виде содержатся в тексте, мы легко находим ответы.

In [10]:
print_search('Кто такой протопоп Аввакум?', 1)

Question Кто такой протопоп Аввакум?
Top1 Протопоп Аввакум, духовный отец старообрядцев, противников изменения церковных обрядов и богослужебных книг в соответствии с новейшими греческими образцами, призывал оказывать сопротивление всем нововведениям пришедшего к власти патриарха Никона, определяя его реформы как отход от православия. 1.0


In [11]:
print_search('Что в основном издает издательство Мельбурнского университета?', 1)

Question Что в основном издает издательство Мельбурнского университета?
Top1 Издательство Мельбурнского университета выпускает преимущественно научные работы, а издательство «Ангус и Робертсон» в Сиднее – многие важные произведения, созданные в Австралии. 0.897878932953


In [12]:
print_search('Какой минимально допустимый возраст для голосования в США?', 1)

Question Какой минимально допустимый возраст для голосования в США?
Top1 26-я поправка к Конституции, вступившая в силу в 1971, установила минимальный возраст избирателя – 18 лет для всех выборов на уровне штата и государства в целом. 0.691259896755


Иногда правильное предложение находится не на первой позиции:

In [13]:
print_search('Какое место занимает Россия по добыче газа?', 2)

Question Какое место занимает Россия по добыче газа?
Top1 Топливно-энергетический комплекс (ТЭК) РФ объединяет также отрасли, осуществляющие добычу, транспортировку, обработку первичных энергетических ресурсов (нефть, газ, уголь), занимая 2-е место в мире после США по производству энергоресурсов и 3-е место в мире после США и Китая по внутреннему употреблению. 0.899263077974
Top2 РФ занимает первое место в мире по запасам газа (32% мировых запасов, 30% мировой добычи); второе место по уровню добычи нефти (348,8 млн. 0.899263077974


Пример плохой работы: Брюсселя нет в топ-5.

In [14]:
print_search('Какой город столица Бельгии?', 5)

Question Какой город столица Бельгии?
Top1 В 1996 в Бельгии насчитывалось 13 городов с населением более 65 тыс. 0.890823185444
Top2 ), третий по численности населения город Германии и столица Баварии. 0.867529948552
Top3 – портового города, второго в стране по числу жителей после Берлина; Бремена – еще одного портового города; Ганновера, столицы земли Нижняя Саксония; Нюрнберга, оставившего яркий след в истории Германии и сохранившего многочисленные памятники Средневековья; а также конурбация Саарланд. 0.867529948552
Top4 В послевоенный период, с потерей Берлина как общенациональной столицы, ее функции фактически оказались распределены между несколькими городами Западной Германии, что стало одной из самых характерных черт развития городской сети ФРГ. 0.867529948552
Top5 В город стекались люди искусства и торговцы со всей Австрии, из Чехии и Венгрии, из Испании и Нидерландов, из Италии и Южной Германии. 0.812884032726


Модель хорошо "обходит" синонимы. Пример:

In [15]:
print_search('Кто доминирует в населении Бразилии?', 1)

Question Кто доминирует в населении Бразилии?
Top1 В составе населения Бразилии преобладает молодежь, и этим объясняются высокие темпы рождаемости. 0.914728124936


Слово "доминирует" было успешно заменено на "преобладает", ведь оно есть в списке синонимов:

In [16]:
searcher.w2v_model.wv.most_similar('доминировать_VERB')

[('преобладать_VERB', 0.7441843748092651),
 ('доминирующий_ADJ', 0.7359710931777954),
 ('превалировать_VERB', 0.6631802916526794),
 ('доминирование_NOUN', 0.6425091624259949),
 ('главенствовать_VERB', 0.5751291513442993),
 ('господствовать_VERB', 0.566472053527832),
 ('вытеснять_VERB', 0.5470854640007019),
 ('сформировываться_VERB', 0.5379319190979004),
 ('лидировать_VERB', 0.5320693850517273),
 ('ассоциироваться_VERB', 0.5176834464073181)]

### Поиск сущностей с помощью texterra

После того, как был реализован поиск релевантных предложений, была построена модель поиска сущностей в предложениях. Для этого использовалась готовая реализация из библиотеки texterra, которая дает доступ к API одноименного веб-приложения.

In [17]:
import texterra
t = texterra.API('API-KEY')

Протестируем работу модуля. Как видим, были успешно найдены сущности "человек" и "город". Хотя в документации библиотеки и было заявлено, что сущности типа "дата и время" тоже поддерживаются, нам не удалось их найти (см. пример)

In [18]:
for i in t.named_entities('Иванов Иван Иванович приехал в Москву 15 ноября 2017 года в 2:00'):
    print(i)

[(0, 20, 'Иванов Иван Иванович', 'PERSON'), (31, 37, 'Москву', 'GPE_CITY')]


Однако, хорошо определяются сущности географических объектов: страны, регионы, реки и т.д

In [19]:
for i in t.named_entities('Самая большая река Германии – Рейн, питающийся от огромного веера притоков в предгорьях Альп.'):
    print(i)

[(19, 27, 'Германии', 'GPE_COUNTRY'), (30, 34, 'Рейн', 'LOCATION_RIVER'), (88, 92, 'Альп', 'LOCATION_OTHER')]


Напишем функцию, которая возвращает словарь с найденными сущностями заданных типов для нескольких предложений с их рейтингами из поиска. Таким образом, из предложений-кандидатов извлекаются сущности, которые могут являться ответом на вопрос. Для каждого слова-кандидата подсчитывается его рейтинг на основе рейтинга предложения, в котором это слово содержится. Если слово встречается несколько раз, то рейтинги суммируются.

Затем было решено использовать квадрат рейтинга предложения, чтобы сильнее наказывать за менее релевантные предложения.

In [20]:
ms = Mystem()

def entities_extraction(sents, scores, entfilt):
    exctracted = t.named_entities(sents)
    ents_dict = {}
    for i, res in enumerate(exctracted):
        for entity in res:
            if entity[-1] in entfilt:
                word = ' '.join([w['analysis'][0]['lex'] for w in ms.analyze(entity[2])\
                                if 'analysis' in w and len(w['analysis'])])
                weight = scores[i] ** 2
                if word in ents_dict.keys():
                    ents_dict[word] += weight
                else:
                    ents_dict[word] = weight
    return ents_dict

Пример запроса. Топ-5 предложений

In [22]:
q = 'В какой стране город Сидней?'
print_search(q, 5)

Question В какой стране город Сидней?
Top1 Австралия обладает большей частью мировых запасов рутила, циркона и тория, содержащихся в песках вдоль восточного побережья страны между островом Страдброк (Квинсленд) и городом Байрон-Бей (Новый Южный Уэльс) и на побережье Западной Австралии в Кейпеле. 0.88725656271
Top2 Столица страны Канберра была спланирована как образцовый город Уолтером Берли Гриффином в 1911. 0.877177218596
Top3 км), где находится город Канберра – столица страны и резиденция правительства. 0.877177218596
Top4 Институты законодательной, судебной и исполнительной власти федерального правительства Союза сосредоточены в столице страны Канберре. 0.76800040404
Top5 В настоящее время золото добывают во многих районах страны, но преимущественно в Западной Австралии. 0.737295528253


Извлеченные ответы с весами:

In [23]:
entities_extraction(*searcher.get_relevant_sentences(q, n=5), ['GPE_COUNTRY'])

{'австралия': 0.78722420807162408,
 'западный австралия': 0.54360469598175165,
 'страна': 1.3130445688058128}

Видно, что слово "страна" имеет лучший рейтинг, но оно явно не подходит. Очевидно, нужно убирать из рассмотрения слова, содержащиеся в запросе.

Функция ниже обрабатывает вопрос, производит для него поиск и извлекает сущности заданного типа.

In [24]:
def extract_answer(q, entfilt, n):
    sents, scores = searcher.get_relevant_sentences(q, n=n)
    if not entfilt:
        return None, sents[0]
    candidates = entities_extraction(sents, scores, entfilt).items()
    if not len(candidates):
        return None, sents[0]
    words_in_q = [w['analysis'][0]['lex'] for w in ms.analyze(q) if 'analysis' in w]
    candidates = [c for c in candidates if c[0] not in words_in_q]
    candidates.sort(key=lambda tup: tup[1], reverse=True)
    return candidates[0][0], sents[0]

Ответ для предыдущего запроса:

In [25]:
extract_answer(q, ['GPE_COUNTRY'], 5)[0]

'австралия'

Продолжаем тестировать:

In [26]:
extract_answer('В какой стране город Москва?', ['GPE_COUNTRY'], 5)[0]

'россия'

К сожалению, ошибки все равно возникают

In [27]:
extract_answer('В какой стране Москва?', ['GPE_COUNTRY'], 5)[0]

'украина'

### Шаблоны типов вопросов и окончательный алгоритм

Теперь необходимо научиться определять тип вопросов. Использовались шаблоны, представленные в таблице ниже. Проверялось вхождение шаблона в строку вопроса:

In [28]:
parser_table = [{'question': 'где', 'entities': ['GPE_COUNTRY', 'GPE_CITY', 'GPE_STATE_PROVINCE', 'GPE_OTHER', 'LOCATION_REGION', 'LOCATION_CONTINENT', 'LOCATION_OTHER']},{'question': 'кто', 'entities': ['PERSON']},{'question': 'какой страна', 'entities': ['GPE_COUNTRY']},{'question': 'какой город', 'entities': ['GPE_CITY']},{'question': 'какой река', 'entities': ['LOCATION_RIVER']},{'question': 'какой организация', 'entities': ['FACILITY', 'ORGANIZATION_CORPORATION', 'ORGANIZATION_EDUCATIONAL', 'ORGANIZATION_POLITICAL']},{'question': 'какой компания', 'entities': ['FACILITY', 'ORGANIZATION_CORPORATION', 'ORGANIZATION_EDUCATIONAL', 'ORGANIZATION_POLITICAL']},{'question': 'какой национальность', 'entities': ['NORP_NATIONALITY', 'LANGUAGE']},{'question': 'какой религия', 'entities': ['NORP_RELIGION']}]

import pandas as pd
pd.set_option('max_colwidth',1000)
parser_table = pd.DataFrame.from_records(parser_table, index='question')
parser_table

,entities
question,
где,"[GPE_COUNTRY, GPE_CITY, GPE_STATE_PROVINCE, GPE_OTHER, LOCATION_REGION, LOCATION_CONTINENT, LOCATION_OTHER]"
кто,[PERSON]
какой страна,[GPE_COUNTRY]
какой город,[GPE_CITY]
какой река,[LOCATION_RIVER]
какой организация,"[FACILITY, ORGANIZATION_CORPORATION, ORGANIZATION_EDUCATIONAL, ORGANIZATION_POLITICAL]"
какой компания,"[FACILITY, ORGANIZATION_CORPORATION, ORGANIZATION_EDUCATIONAL, ORGANIZATION_POLITICAL]"
какой национальность,"[NORP_NATIONALITY, LANGUAGE]"
какой религия,[NORP_RELIGION]


Парсер для вопросов:

In [29]:
def get_q_type(q):
    parsed_q = ' '.join([w['analysis'][0]['lex'] for w in ms.analyze(q) \
                         if 'analysis' in w and len(w['analysis'])])
    for i in parser_table.index:
        if i in parsed_q:
            return parser_table.loc[i].entities
    return None

Пример классификации:

In [30]:
get_q_type('Где протекает река Днепр?')

['GPE_COUNTRY',
 'GPE_CITY',
 'GPE_STATE_PROVINCE',
 'GPE_OTHER',
 'LOCATION_REGION',
 'LOCATION_CONTINENT',
 'LOCATION_OTHER']

В результате система возвращает ответ (precise), если вопрос известного типа. В любом случае мы также возвращаем самое релевантное предложение из поиска. По умолчанию ответ ищется в первых пяти предложениях.

In [31]:
def qa(q, n=5):
    entfilt = get_q_type(q)
    answer, top1 = extract_answer(q, entfilt, n)
    print('\033[1mQuestion\033[0m', q)
    print('\033[1mPrecise\033[0m', answer)
    print('\033[1mTop1\033[0m', top1)

### Примеры работы

Приводим примеры успешной работы модели на каждый тип вопроса.

In [32]:
qa('Где протекает река Днепр')

Question Где протекает река Днепр
Precise украина
Top1 Основные реки Украины – Днепр, Южный Буг и Дунай, впадающие в Черное море.


In [33]:
qa('Кто присоединил Крым к России')

Question Кто присоединил Крым к России
Precise екатерина
Top1 Российская Федерация, как правопреемник СССР оставалась владельцем практически всей собственности советского государства, а также всех долгов СССР; в то же время некоторые российские политики требовали присоединения к России Донбасса и Крыма (последний был  передан Украине из состава РСФСР Н.С.Хрущевым в 1954 фактически самовольно на основании указа Президиума Верховного Совета СССР).


In [34]:
qa('Кто возглавлял белую армию?')

Question Кто возглавлял белую армию?
Precise а и деникин
Top1 К концу лета 1918 белое движение, состоявшие из офицеров царской армии, юнкеров, казаков, представителей дворянства и руководимое адмиралом А.В.Колчаком в Западной Сибири, генералами А.И.Деникиным на юге и Н.Н.Юденичем на северо-западе (Прибалтика), оказалось поддержанным иностранной интервенцией.


In [35]:
qa('В какой стране притекает река Ганг?')

Question В какой стране притекает река Ганг?
Precise бангладеш
Top1 На юге страны р.Кавери обладает многими удобными створами для строительства ГЭС.


In [36]:
qa('В каком городе находится музей Лувр?')

Question В каком городе находится музей Лувр?
Precise париж
Top1 Города стали центрами культурной и духовной жизни страны, здесь находились университеты, научно-исследовательские центры, медицинские колледжи при больницах, музеи и художественные галереи, оркестры и концертные залы, театры и издательства.


In [37]:
qa('В какую организацию входят РФ и Китай?')

Question В какую организацию входят РФ и Китай?
Precise оон
Top1 В созданную в 2001 Шанхайскую организацию сотрудничества вошли РФ, Китай, Казахстан, Киргизия, Таджикистан и Узбекистан.


In [38]:
qa('Какая религия преобладает в России')

Question Какая религия преобладает в России
Precise христианство
Top1 Преобладающая часть религиозного населения РФ – православные, в том числе принадлежащие к сектам и группам православной направленности (старообрядцы, молокане).


In [39]:
qa('Кто президент России?')

Question Кто президент России?
Precise в путин
Top1 Президент России В.Путин подписал указ о признании республики Крым как независимого и суверенного государства и особого статуса города Севастополя в составе независимого Крыма.


In [40]:
qa('Какая река самая длинная в Бразилии')

Question Какая река самая длинная в Бразилии
Precise амазонка
Top1 По ресурсам водного стока Россия занимает 2-е место в мире после Бразилии с ее самой многоводной рекой Амазонкой.


In [41]:
qa('Какой город самый крупный в Канаде?')

Question Какой город самый крупный в Канаде?
Precise торонто
Top1 Во всех крупных городах Канады есть симфонические оркестры, самыми значительными считаются оркестры Торонто, Монреаля и Ванкувера.


Ошибки возникают в основном в связи со следующей причиной: каждое слово в вопросе имеет одинаковый вес в поиске, но на некоторые слова нужно обращать особое внимание (название страны в вопросе про столицу, например), то есть выделять сущности следует также в запросе во время поиска. Пример с Москвой и Украиной выше это показывает.

### Оценка качества

Мы тестировали систему на данных из соревнования Сбербанка http://contest.sdsj.ru Данные представляют собой размеченные фрагменты текста с вопросом и ответом. Мы протестировали систему на 100 образцах, вычисляли число верно найденных предложений и верных ответов, но только для вопросов, которые умеем распознавать.

In [13]:
def is_answer(answer, label):
    if answer:
        parsed_label = ' '.join([w['analysis'][0]['lex'] for w in ms.analyze(label)\
                                 if 'analysis' in w and len(w['analysis'])])
        return answer in parsed_label
    return False

Загрузка данных:

In [15]:
test = pd.read_csv('train_task_b.csv')
test.shape

(50364, 5)

In [16]:
test.head(1)

,paragraph_id,question_id,paragraph,question,answer
0,14754,60544,"Первые упоминания о строении человеческого тела встречаются в Древнем Египте. В XXVII веке до н. э. египетский врач Имхотеп описал некоторые органы и их функции, в частности головной мозг, деятельность сердца, распространение крови по сосудам. В древнекитайской книге Нейцзин (XI—VII вв. до н. э.) упоминаются сердце, печень, лёгкие и другие органы тела человека. В индийской книге Аюрведа ( Знание жизни , IX-III вв. до н. э.) содержится большой объём анатомических данных о мышцах, нервах, типах телосложения и темперамента, головном и спинном мозге.",Где встречаются первые упоминания о строении человеческого тела?,в Древнем Египте


In [17]:
test_idx = []
for idx in test.index:
    if get_q_type(test.loc[idx, 'question']):
        test_idx.append(idx)

In [18]:
test_df = test.loc[test_idx]

In [19]:
test_df.shape

(7748, 5)

Запускаем тестирование

In [20]:
searcher = QA_word2vec(' '.join(test_df['paragraph']))

In [43]:
searcher.ms = Mystem()

In [21]:
acc_sent = 0
acc_answers = 0
cnt_none = 0
for i, idx in enumerate(test_df.index):
    try:
        q = test_df.loc[idx, 'question']
        entfilt = get_q_type(q)
        answer, top1 = extract_answer(q, entfilt, 1)
        
        if top1 in test_df.loc[idx, 'paragraph']:
            acc_sent += 1
        if answer:
            if is_answer(answer, test_df.loc[idx, 'answer']):
                acc_answers += 1
        else:
            cnt_none += 1
    except:
        pass
        
    if i % 10 == 0:
        print((i, acc_sent, acc_answers, cnt_none), end=' ')

(0, 1, 1, 0) (10, 9, 3, 6) (20, 17, 5, 10) (30, 23, 7, 13) (40, 23, 7, 13) (50, 30, 12, 15) (60, 39, 17, 19) (70, 46, 21, 21) (80, 54, 26, 24) (90, 64, 30, 28) (100, 72, 33, 32)

Из 100 объектов тестовой выборки в 72 система правильно нашла предложение с ответом, в 33 случаях правильно был найден сам ответ.

В 32 случаях система не смогла вернуть хоть какую-либо сущность (вернула None), это указывает на ошибки в распознавании типа ответа (он очевидно очень ограничен), либо на ошибки модуля поиска сущностей.

Значит, что в 48% случаев из тех, когда сущности находились, был получен правильный ответ, что неплохо. Наша модель не учитывает тонких синтаксических зависимостей и контекст, а просто подсчитывает количество найденных сущностей с учетом релевантности предложения.